In [1]:
import pandas as pd
#data = pd.read_csv('C:/Users/Carlos/Downloads/earthquake_spain_19002024.csv', header=0)

In [2]:
data_df = pd.read_csv('C:/Users/Carlos/Downloads/earthquake_spain_19002024.csv', delimiter=';', skipinitialspace=True)

In [11]:
data_df.head()

,Evento,Fecha,Hora,Latitud,Longitud,Prof. (Km),Inten.,Mag.,Tipo Mag.,Localización
0,2494,16/02/1900,05:00:00,43.0,0.1500,0.0,III,NaN,NaN,Bagneres-de-Bigorre.FRA
1,2495,04/03/1900,09:00:00,40.0,3.8333,0.0,IV,NaN,NaN,Ciutadella.IB
2,2496,09/03/1900,04:20:00,37.6,-1.3167,0.0,IV,NaN,NaN,Mazarrón.MU
3,2497,25/04/1900,18:00:00,35.7,-0.6000,0.0,VI,NaN,NaN,ORAN.ARG
4,2498,09/06/1900,23:50:00,42.6,-5.5667,0.0,III,NaN,NaN,León


In [3]:
data_df = data_df.drop(['Evento'], axis=1)

In [4]:
data_df = data_df.drop(['Hora'], axis=1)

In [5]:
data_df = data_df.drop(['Fecha'], axis=1)

In [6]:
data_df = data_df.drop(['Mag.'], axis=1)

In [7]:
data_df = data_df.drop(['Tipo Mag.'], axis=1)

In [8]:
data_df = data_df.drop(['Prof. (Km)'], axis=1)

In [9]:
data_df_sinloc = data_df.drop(['Localización'], axis=1)

In [10]:
data_df_sinloc.head()

,Latitud,Longitud,Inten.
0,43.0,0.1500,III
1,40.0,3.8333,IV
2,37.6,-1.3167,IV
3,35.7,-0.6000,VI
4,42.6,-5.5667,III


In [14]:
import plotly.express as px
import plotly.io as pio

In [15]:
pio.renderers.default = "browser"

In [16]:
fig = px.density_mapbox(data_df_sinloc, lat='Latitud', lon='Longitud', radius=10,
                        center=dict(lat=40, lon=-3), zoom=5,
                        mapbox_style='stamen-terrain')

In [21]:
fig2 = px.density_mapbox(data_df_sinloc, lat='Latitud', lon='Longitud', radius=10,
                        center=dict(lat=40, lon=-3), zoom=5,
                        mapbox_style='carto-positron')

In [19]:
#pio.show(fig)

In [20]:
fig.write_html("mapa.html")

In [24]:
fig2.write_html("mapa2.html")

In [25]:
data_df

,Latitud,Longitud,Inten.,Localización
0,43.0000,0.1500,III,Bagneres-de-Bigorre.FRA
1,40.0000,3.8333,IV,Ciutadella.IB
2,37.6000,-1.3167,IV,Mazarrón.MU
3,35.7000,-0.6000,VI,ORAN.ARG
4,42.6000,-5.5667,III,León
...,...,...,...,...
187842,38.1579,-1.4956,NaN,SW ABARÁN.MU
187843,42.8588,-7.3230,NaN,SW BARALLA.LU
187844,38.2793,0.6721,NaN,MEDITERRÁNEO-CABO DE PALOS
187845,34.7910,-4.3519,NaN,SE ZERKAT.MAC
